# Flame

In order to download the models we need to look at: https://flame.is.tue.mpg.de/download.php
This could be usefull if onw want's to look how to load the FLAME model from the SMPL loader: https://github.com/Rubikplayer/flame-fitting/blob/master/smpl_webuser/serialization.py#L117
Useful utils if one needs to transform the chumpy format into nupy or torch: https://github.com/vchoutas/smplx/blob/main/smplx/utils.py

NOTE: That if one want't to unpickle old python=2.x numpy code, we need to use the encoding="latin1". For more information please refere to: https://docs.python.org/3/library/pickle.html


In [ ]:
from pathlib import Path
import numpy as np
from lib.flame.utils import load_flame
from lib.flame.model import FLAME

# https://github.com/soubhiksanyal/FLAME_PyTorch/blob/master/flame_pytorch/flame.py
data_dir = Path("/Users/robinborth/Code/GuidedResearch/data/dphm_christoph_mouthmove")
flame_dir = "/Users/robinborth/Code/GuidedResearch/checkpoints/flame2023"
flame_dict = load_flame(flame_dir)
flame_model = FLAME(flame_dir=flame_dir)
print("FLAME keys:")
print(list(flame_dict.keys()))
print()

# This is the linear blend skinning (LBS) with corrective blendshapes with N=5023 and
# K=4 joint (neck, jaw, and eyeballs (left, right))
bs_style = flame_dict["bs_style"]
print("bs_style:", bs_style)
bs_type = flame_dict["bs_type"]
print("bs_type:", bs_type)

# this is the template mesh, e.g. T bar in the "zero pose"
v_template = flame_dict["v_template"]
print("v_template:", v_template.shape)
for i in range(3):
    d = v_template[:, i].max() - v_template[:, i].min()
    s = ["x", "y", "z"][i]
    print(f"{s}-delta in meter {d:.2}m")

# those are used in the pytorch flame example
f = flame_dict["f"]
print("f:", f.shape)

# shape (beta); note that the dimension is (5023, 3, 400)
# where the first 300 are for the shape params and the last 400 for the expression
# params, but the matrix is shared
shapedirs = flame_dict["shapedirs"]
print("shapedirs:", shapedirs.shape)

# pose (theta)
posedirs = flame_dict["posedirs"]
print("posedirs:", posedirs.shape)

# is this the expressions? (psi)
weights = flame_dict["weights"]  # lbs := linear blend shapes
print("weights:", weights.shape)

# Linear smoothed by skinning function(T, J, theta, W).
# Blendweights W (KxN) are J_regressor
J_regressor = flame_dict["J_regressor"]
print("J_regressor:", J_regressor.shape)

# J are the joints that the vertices of T are rotated
J = flame_dict["J"]
print("J:", J.shape)

kintree_table = flame_dict["kintree_table"]
print("kintree_table:", kintree_table.shape)

# Falme Landmarks

The landmark file defines the barycentric embedding of 105 points of the Mediapipe mesh in the surface of FLAME.
In consists of three arrays: lmk_face_idx, lmk_b_coords, and landmark_indices.

- lmk_face_idx contains for every landmark the index of the FLAME triangle which each landmark is embedded into
- lmk_b_coords are the barycentric weights for each vertex of the triangles
- landmark_indices are the indices of the vertices of the Mediapipe mesh


In [ ]:
from lib.flame.utils import load_static_landmark_embedding

flame_landmarks = load_static_landmark_embedding(flame_dir)
print(list(flame_landmarks.keys()))
print()

print("lmk_face_idx:")
print(flame_landmarks["lmk_face_idx"][:5])
print(flame_landmarks["lmk_face_idx"].min())
print(flame_landmarks["lmk_face_idx"].max())
print(flame_landmarks["lmk_face_idx"].shape)
print()

print("lmk_b_coords:")
print(flame_landmarks["lmk_b_coords"][:5])
print(flame_landmarks["lmk_b_coords"].min())
print(flame_landmarks["lmk_b_coords"].max())
print(flame_landmarks["lmk_b_coords"].shape)
print()

print("landmark_indices:")
print(flame_landmarks["landmark_indices"][:5])
print(flame_landmarks["landmark_indices"].min())
print(flame_landmarks["landmark_indices"].max())
print(flame_landmarks["landmark_indices"].shape)

# FLAME Mask

Dictionary with vertex indices for different masks for the publicly available FLAME head model (https://flame.is.tue.mpg.de/).
See the gif for a visualization of all masks.


In [ ]:
from lib.flame.utils import load_flame_masks

flame_masks = load_flame_masks(flame_dir)
print(list(flame_masks.keys()))
print()

print("eye_region:")
print(flame_masks["eye_region"][:5])
print(flame_masks["eye_region"].min())
print(flame_masks["eye_region"].max())
print(flame_masks["eye_region"].shape)

# FLAME Forward

We use the right hand rule; note that for this rule we have a counter-clockwise.


In [ ]:
import torch

shape_params = torch.zeros(300)
# shape_params = torch.randn(300)
expression_params = torch.zeros(100)
# expression_params = torch.randn(100)

vertices_out = flame_model(
    shape_params=shape_params,
    expression_params=expression_params,
    global_pose=torch.tensor([0.0, 0.0, 0.0]),
    # global_pose=torch.tensor([np.pi, 0.0, 0.0]),
    neck_pose=torch.tensor([0.0, 0.0, 0.0]),
    jaw_pose=torch.tensor([0.0, 0.0, 0.0]),
    eye_pose=torch.tensor([0, 0, 0, 0, 0, 0]),
    transl=torch.tensor([0, 0, 0.0]),
)
vertices = vertices_out.detach().cpu().numpy()

np.save("temp/vertices", vertices)
print("x_min_max", vertices[:, 0].min(), vertices[:, 0].max())
print("y_min_max", vertices[:, 1].min(), vertices[:, 1].max())
print("z_min_max", vertices[:, 2].min(), vertices[:, 2].max())

# Render with Trimesh & Pyrender


In [ ]:
import torch
import numpy as np
import trimesh
from pyrender import OffscreenRenderer, Mesh, Scene, PerspectiveCamera
import matplotlib.pyplot as plt

# convert to trimesh
vertex_colors = np.ones([vertices.shape[0], 4]) * [0.3, 0.3, 0.3, 0.8]
mesh = trimesh.Trimesh(vertices, flame_model.faces, vertex_colors=vertex_colors)

# add mesh to scene
scene = Scene()
scene.add(Mesh.from_trimesh(mesh))

# create camera
cam = PerspectiveCamera(yfov=(np.pi / 3.0))
cam_pose = np.array(
    [
        [1.0, 0.0, 0.0, 0.0],
        [0.0, 1.0, 0.0, 0.0],
        [0.0, 0.0, 1.0, 1.0],
        [0.0, 0.0, 0.0, 1.0],
    ]
)
cam_node = scene.add(cam, pose=cam_pose)

# render the scene
renderer = OffscreenRenderer(viewport_width=1024, viewport_height=1024)
color, depth = renderer.render(scene)
renderer.delete()
plt.figure(figsize=(5, 5))
plt.imshow(color)
plt.show()

# Custom Renderer with Mesh Rasterization


In [ ]:
from lib.camera import camera2pixel
import matplotlib.pyplot as plt

K = dict(fx=512, fy=512, cx=128, cy=128)
lm = camera2pixel(vertices, **K).astype(int)

img = np.ones((1920, 1080))
for point in lm:
    u, v, z = point.astype(int)
    v = -v
    if 0 <= u < 1920 and 0 <= v < 1080:
        img[u, v] = 0

plt.imshow(img)

# Open3D Point Cloud


In [ ]:
import open3d as o3d
import numpy as np

C = 5023
red = [np.array([255, 0, 0], dtype=np.uint8)] * C
red = o3d.utility.Vector3dVector(np.stack(red))
green = [np.array([0, 255, 0], dtype=np.uint8)] * C
green = o3d.utility.Vector3dVector(np.stack(green))
blue = [np.array([0, 0, 255], dtype=np.uint8)] * C
blue = o3d.utility.Vector3dVector(np.stack(blue))

vertices = np.load("temp/vertices.npy")
pcd_flame = o3d.geometry.PointCloud()
pcd_flame.points = o3d.utility.Vector3dVector(vertices)
pcd_flame.colors = blue
o3d.visualization.draw_plotly([pcd_flame])

In [ ]:
import open3d as o3d
from pathlib import Path
from lib.data.utils import load_points_3d

data_dir = Path("/Users/robinborth/Code/GuidedResearch/data/dphm_christoph_mouthmove")
points = load_points_3d(data_dir=data_dir, idx=0)
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
pcd.colors = red
o3d.visualization.draw_plotly([pcd, pcd_flame])

# Pytorch3D Rasterizer

We want to implement our own rasterizer, hence we can look how pytorch metric is doing it:
from pytorch3d.renderer.mesh import rasterize_meshes
Or we can implmenet it, for reference here:
https://www.scratchapixel.com/lessons/3d-basic-rendering/rasterization-practical-implementation/rasterization-stage.html

# Go over the rasterization:

https://www.scratchapixel.com/lessons/3d-basic-rendering/rasterization-practical-implementation/overview-rasterization-algorithm.html


In [ ]:
from pytorch3d.renderer.mesh import rasterize_meshes
from pytorch3d.structures import Meshes

s = 2.0
t = torch.tensor([0.0, 0.0, 3.0])
image_size = 256

fixed_vetices = (vertices_out.clone() * s) + t
fixed_faces = flame_model.faces.clone()
meshes_screen = Meshes(verts=[fixed_vetices], faces=[fixed_faces])

pix_to_face, zbuf, bary_coords, dist = rasterize_meshes(
    meshes_screen,
    image_size=image_size,
    blur_radius=0.0,
    faces_per_pixel=1,
    bin_size=None,
    max_faces_per_bin=None,
    perspective_correct=False,
)
# pix_to_face = pix_to_face.squeeze()
# bary_coords = bary_coords.squeeze()
# dist = dist.squeeze()

# print(dist.min(), dist.max())
# plt.imshow(dist.detach().cpu().numpy())

# Albedo Diffuse


In [ ]:
import numpy as np

path = "/Users/robinborth/Code/GuidedResearch/checkpoints/flame2023_no_jaw/albedoModel2020_FLAME_albedoPart.npz"
albedo = np.load(path)
print(list(albedo.keys()))
print(f"{albedo['vt'].shape=}")
print(f"{albedo['vt'].min()=}")
print(f"{albedo['vt'].max()=}")
print(f"{albedo['ft'].shape=}")
print(f"{albedo['ft'].min()=}")
print(f"{albedo['ft'].max()=}")
print(f"{albedo['specPC'].shape=}")
print(f"{albedo['PC'].shape=}")
print(f"{albedo['specMU'].shape=}")
print(f"{albedo['MU'].shape=}")

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(albedo["MU"])

# Other Albedo


In [ ]:
path = "/Users/robinborth/Code/GuidedResearch/checkpoints/flame2023_no_jaw/FLAME_texture.npz"
albedo = np.load(path)

list(albedo.keys())
print(f"{albedo['vt'].shape=}")
print(f"{albedo['vt'].min()=}")
print(f"{albedo['vt'].max()=}")
print(f"{albedo['ft'].shape=}")
print(f"{albedo['ft'].min()=}")
print(f"{albedo['ft'].max()=}")
print(f"{albedo['tex_dir'].shape=}")
print(f"{albedo['mean'].shape=}")

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(albedo["mean"].astype(int))